In [28]:
from dataloader import get_dataloader
import os

## DataSet

In [23]:
batch_size = 2
shuffle_img = False
num_workers = 2
split = "train"

In [26]:

dataloader_train = get_dataloader(batch_size=batch_size,shuffle=shuffle_img,num_workers=num_workers,split=split)

Using downloaded and verified file: ./flickr30k/flickr30k_train.json


In [29]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"
img  = next(iter(dataloader_train))

In [30]:
batch

(tensor([[[[-1.4065, -1.4478, -1.2965,  ...,  1.8958,  0.1759, -1.2965],
           [-1.4753, -1.4478, -1.3653,  ...,  1.9234,  1.9509,  1.6894],
           [-1.4341, -1.4203, -1.3928,  ...,  1.7720,  1.8958,  1.9509],
           ...,
           [-0.2645, -1.5029, -0.5534,  ...,  0.3960, -0.9249, -0.8424],
           [-0.0993, -1.2277, -0.8561,  ..., -0.8837, -0.5121, -0.1681],
           [-0.2369, -1.5579, -1.5579,  ...,  0.0520,  0.3547,  1.4968]],
 
          [[-1.1093, -1.0385, -0.8261,  ...,  2.0619,  0.3772, -1.0244],
           [-1.3500, -0.9960, -0.7978,  ...,  2.0336,  2.0478,  1.7929],
           [-1.3500, -1.0244, -0.7695,  ...,  2.0478,  2.0619,  2.0761],
           ...,
           [ 0.3489, -0.9111,  0.1790,  ...,  0.1224, -0.9111, -0.7695],
           [ 0.4763, -0.4722, -0.3873,  ..., -0.8686, -0.5288,  0.1082],
           [ 0.2356, -1.2933, -1.2650,  ...,  0.1507,  0.2073,  1.5240]],
 
          [[-1.2502, -1.2776, -1.0582,  ...,  2.1503,  0.3130, -1.0034],
           [-

## Model

In [11]:
from transformers import BertTokenizerFast,ViTFeatureExtractor, BertModel, ViTModel

In [9]:
vision_model_name = "facebook/dino-vits16"
text_model_name = "prajjwal1/bert-medium"

In [10]:
tokenizer = BertTokenizerFast.from_pretrained(text_model_name)
feature_extractor = ViTFeatureExtractor.from_pretrained(vision_model_name)

In [14]:
vision_model = ViTModel.from_pretrained(vision_model_name)
#vision_model.config

Some weights of ViTModel were not initialized from the model checkpoint at facebook/dino-vits16 and are newly initialized: ['pooler.dense.weight', 'pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
text_model = BertModel.from_pretrained(text_model_name)
text_model.config

Downloading: 100%|██████████| 159M/159M [00:02<00:00, 82.8MB/s] 
Some weights of the model checkpoint at prajjwal1/bert-medium were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertConfig {
  "_name_or_path": "prajjwal1/bert-medium",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 512,
  "initializer_range": 0.02,
  "intermediate_size": 2048,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 8,
  "num_hidden_layers": 8,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [17]:
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = text_model(**inputs)

last_hidden_states = outputs.last_hidden_state

In [18]:
inputs

{'input_ids': tensor([[  101,  7592,  1010,  2026,  3899,  2003, 10140,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

In [20]:
last_hidden_states

tensor([[[ 0.5313, -0.0959,  1.1669,  ..., -0.4262,  0.4165, -0.2875],
         [ 0.0024, -0.7301,  0.5934,  ..., -0.4460,  1.2140,  0.8000],
         [-0.2985,  0.1856,  0.4116,  ..., -0.0307,  0.8842, -0.3856],
         ...,
         [ 0.0987,  0.4613,  0.8336,  ..., -0.5302,  0.4894,  0.3599],
         [ 0.2055,  0.2889,  1.0970,  ..., -0.1850,  0.6020, -0.0864],
         [ 0.3357, -0.0997,  1.6814,  ..., -0.6950,  1.0993,  0.0910]]],
       grad_fn=<NativeLayerNormBackward0>)